get the log data

In [ ]:
import pandas
import matplotlib as plt
import numpy as np

scale_factor = 1000000

pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_colwidth", None)
frames = pandas.read_csv("../../install/bin/sc2019_profiling_log_st.csv", sep=";", quotechar="'")
raw_frames = pandas.read_csv("../../install/bin/sc2019_profiling_log.csv", sep=";", quotechar="'")

random sample

In [ ]:
sample_row = 16753
print(frames.loc[sample_row])
print(raw_frames.loc[sample_row])

scaling

In [ ]:
frames["start"] = frames["start"].div(scale_factor)
frames["end"] = frames["end"].div(scale_factor)
frames["total_time"] = frames["total_time"].div(scale_factor)
frames["self_time"] = frames["self_time"].div(scale_factor)
raw_frames["duration (ns)"] = raw_frames["duration (ns)"].div(scale_factor)
raw_frames["start (ns)"] = raw_frames["start (ns)"].div(scale_factor)
raw_frames["end (ns)"] = raw_frames["end (ns)"].div(scale_factor)

In [ ]:
print(frames.loc[sample_row])
print(raw_frames.loc[sample_row])

# scratchpad: inspect a single frame

In [ ]:
detail_frame = frames.loc[(frames["frame"] == 1190) & (frames["parent"] == "::View3DGL_1::rendering->::RaycastVolumeRenderer1::rendering")]
detail_frame

In [ ]:
detail_raw_frame = raw_frames.loc[(frames["frame"] == 1190) & (frames["parent"] == "::Left-Right::render2->::View3DGL_1::render")]
detail_raw_frame

find the render calls

In [ ]:
gpu = frames.loc[(frames["parent"] == "::AnimationRenderer_1::observe->::ArrowRenderer1::rendering") & (frames["name"] == "Render") & (frames["api"] == "OpenGL") & (frames["frame"] > 0)]
cpu = frames.loc[(frames["parent"] == "::AnimationRenderer_1::observe->::ArrowRenderer1::rendering") & (frames["name"] == "Render") & (frames["api"] == "CPU") & (frames["frame"] > 0)]

f'gpu: {gpu.shape[0]} cpu: {cpu.shape[0]}'

In [ ]:
f'min: {gpu["self_time"].min() / 1000000} (at {gpu["self_time"].idxmin()}) max: {gpu["self_time"].max() / 1000000} (at {gpu["self_time"].idxmax()}) mean: {gpu["self_time"].mean() / 1000000}'

try to find out how values are distributed

In [ ]:
gpu["self_time"].hist(bins=20)
#plt.pyplot.hist(gpu["self_time"], bins=20)
#plt.pyplot.hist(gpu["self_time"], bins=np.arange(0.0,8.0,0.2))

# anomaly: view3dgl seems expensive

In [ ]:
gpu = frames.loc[(frames["parent"] == "::Left-Right::render2->::View3DGL_1::render") & (frames["name"] == "Render") & (frames["api"] == "OpenGL") & (frames["frame"] == 1190)]
cpu = frames.loc[(frames["parent"] == "::Left-Right::render2->::View3DGL_1::render") & (frames["name"] == "Render") & (frames["api"] == "CPU") & (frames["frame"] == 1190)]

f'gpu: {gpu.shape[0]} cpu: {cpu.shape[0]}'

In [ ]:
f'min: {gpu["self_time"].min() / 1000000} (at {gpu["self_time"].idxmin()}) max: {gpu["self_time"].max() / 1000000} (at {gpu["self_time"].idxmax()}) mean: {gpu["self_time"].mean() / 1000000}'

so expensive. why?

In [ ]:
detail = raw_frames.loc[(frames["frame"] == 1190) & (frames["parent"] == "::Left-Right::render2->::View3DGL_1::render")]
detail

In [ ]:
gpu["self_time"].hist(bins=20)

# anomaly: why is view3dgl_1 called twice in frame 1190? (not anymore)

In [ ]:
detail_frame = frames.loc[(frames["frame"] == 1190) & (frames["parent"] == "::Left-Right::render2->::View3DGL_1::render")]
detail_frame

# answer: because we had an old project that had two active views at the same time

(immediately visible in the call details)

# anomaly: why is there two ::View3DGL_1::rendering->::RaycastVolumeRenderer1::rendering	GetExtents	per frame?

In [ ]:
detail_raw_frame = raw_frames.loc[(frames["frame"] == 1190) & (frames["parent"] == "::View3DGL_1::rendering->::RaycastVolumeRenderer1::rendering")]
detail_raw_frame

## but the splitview only calls it once

In [ ]:
detail_raw_frame = raw_frames.loc[(frames["frame"] == 1190) & (frames["parent"] == "::Left-Right::render2->::View3DGL_1::render")]
detail_raw_frame

# answer: for some reason the views all call getextents twice

this might just be a bug

# why are late frames cheaper than middle ones?

ex: 1465 vs. 1832

guess: you see less arrows on the bottom right view

in the graph diff it looks like raycast and arrows got more expensive...? should be the other way around

# why is frame 611 so crazy expensive?

the graph does not seem to say?

# hiccups

ex: arrowrenderer at frame 1199. why? rest of the graph is not affected as much. splom and parco also get a bit slower though